In [10]:
! pip install diffusers
! pip install SpeechRecognition pyaudio
! pip uninstall googletrans
! pip install googletrans-temp


from diffusers import AudioLDM2Pipeline
import speech_recognition as sr
from googletrans import Translator
import torch
import gradio as gr

# import intel_extension_for_pytorch as ipex

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
def translate_text(text, target_language='en'):
    translator = Translator()
    translated_text = translator.translate(text, dest=target_language)
    return translated_text.text

In [ ]:
r = sr.Recognizer()

with sr.Microphone() as source:
    print("請開始說話...")
    audio = r.listen(source)
prompt = 

try:
    # 步驟1：辨識並翻譯音訊
    text = '好聽音樂' #r.recognize_google(audio, language='zh-TW')
    print("您說的是：" + text)
    
    english_translation = translate_text(text)
    print("翻譯結果為：", english_translation)
    prompt = english_translation
   
except sr.UnknownValueError:
    print("無法辨識您的語音")
except sr.RequestError as e:
    print("無法連線至 Google 語音識別服務：{0}".format(e))

In [3]:
model_id = "cvssp/audioldm2"

pipe = AudioLDM2Pipeline.from_pretrained(
    model_id,
    #torch_dtype = torch.float16,
).to("cpu")

generator = torch.Generator("cpu").manual_seed(0)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Loading pipeline components...: 100%|██████████| 11/11 [00:12<00:00,  1.18s/it]


In [4]:
audio = pipe(
    prompt,
    audio_length_in_s = 10.24,
    generator = generator,
).audios[0]

from IPython.display import Audio
Audio(audio, rate = 16000)

  2%|▏         | 3/200 [00:07<08:21,  2.54s/it]


KeyboardInterrupt: 

In [ ]:
def create_music(prompt):
  negative_prompt = "Low quality"
  audio = pipe(
    prompt,
    negative_prompt = negative_prompt,
    audio_length_in_s = 10.24,
    generator = generator,
  ).audios[0]
  return 16000, audio

negative_prompt = "Low quality"

audio = pipe(
    prompt,
    negative_prompt = negative_prompt,
    audio_length_in_s = 10.24,
    generator = generator,
).audios[0]

from IPython.display import Audio
Audio(audio, rate = 16000)

In [ ]:
interface = gr.Interface(
    title = "Music Generation App",
    examples = ["A cheerful ukulele strumming in a beachside jam."],
    fn = create_music,
    inputs = gr.Textbox(),
    outputs = "audio",
).launch(debug=True, share=True)